In [3]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy import inspect
from sqlalchemy import select
from sqlalchemy import text

In [5]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [9]:
inspector = inspect(engine)

In [10]:
print(inspector.get_table_names())

['Bookings', 'Facilities', 'Members']


In [13]:
for table_name in inspector.get_table_names():
    print('Column information for table {}'.format(table_name))
    for info in inspector.get_columns(table_name):
        print(info['name'], info['type'])
    print('-'*40)    

Column information for table Bookings
bookid INTEGER
facid INTEGER
memid INTEGER
starttime VARCHAR(19)
slots INTEGER
----------------------------------------
Column information for table Facilities
facid INTEGER
name VARCHAR(15)
membercost DECIMAL(2, 1)
guestcost DECIMAL(3, 1)
initialoutlay INTEGER
monthlymaintenance INTEGER
----------------------------------------
Column information for table Members
memid INTEGER
surname VARCHAR(17)
firstname VARCHAR(9)
address VARCHAR(39)
zipcode INTEGER
telephone VARCHAR(14)
recommendedby VARCHAR(2)
joindate VARCHAR(19)
----------------------------------------


C:\Users\lmaow\anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:2184: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['5']; using no arguments.
  util.warn(


In [7]:
con = engine.connect()

Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [28]:
query = 'SELECT name, total_revenue '\
'FROM (Select f.name, (SUM(CASE WHEN book.memid=0 THEN f.guestcost*book.slots ELSE f.membercost*book.slots END)-'\
+"COUNT(DISTINCT strftime('%m', book.starttime))*f.monthlymaintenance) AS total_revenue "\
+'FROM Bookings AS book '\
+'LEFT JOIN Facilities AS f '\
+'ON book.facid=f.facid ' \
+'GROUP BY f.name '\
+'ORDER BY total_revenue) AS revenue_table '\
+'WHERE total_revenue<1000'

In [29]:
res = con.execute(query)
res.fetchall()

[('Table Tennis', 150), ('Snooker Table', 195), ('Pool Table', 225)]

Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [32]:
query = 'SELECT DISTINCT m.surname ||" "|| m.firstname AS membername, m1.membername AS recommendername'\
+' FROM Members AS m'\
+' LEFT JOIN (SELECT DISTINCT memid, surname ||" "|| firstname AS membername FROM Members) AS m1'\
+' ON m.recommendedby=m1.memid'\
+' ORDER BY membername'

In [33]:
res = con.execute(query)
res.fetchall()

[('Bader Florence', 'Stibbons Ponder'),
 ('Baker Anne', 'Stibbons Ponder'),
 ('Baker Timothy', 'Farrell Jemima'),
 ('Boothe Tim', 'Rownam Tim'),
 ('Butters Gerald', 'Smith Darren'),
 ('Coplin Joan', 'Baker Timothy'),
 ('Crumpet Erica', 'Smith Tracy'),
 ('Dare Nancy', 'Joplette Janice'),
 ('Farrell David', None),
 ('Farrell Jemima', None),
 ('GUEST GUEST', None),
 ('Genting Matthew', 'Butters Gerald'),
 ('Hunt John', 'Purview Millicent'),
 ('Jones David', 'Joplette Janice'),
 ('Jones Douglas', 'Jones David'),
 ('Joplette Janice', 'Smith Darren'),
 ('Mackenzie Anna', 'Smith Darren'),
 ('Owen Charles', 'Smith Darren'),
 ('Pinker David', 'Farrell Jemima'),
 ('Purview Millicent', 'Smith Tracy'),
 ('Rownam Tim', None),
 ('Rumney Henrietta', 'Genting Matthew'),
 ('Sarwin Ramnaresh', 'Bader Florence'),
 ('Smith Darren', None),
 ('Smith Jack', 'Smith Darren'),
 ('Smith Tracy', None),
 ('Stibbons Ponder', 'Tracy Burton'),
 ('Tracy Burton', None),
 ('Tupperware Hyacinth', None),
 ('Worthington-Sm

Q12: Find the facilities with their usage by member, but not guests

In [38]:
query = 'SELECT f.name, COUNT(*)'\
' FROM Bookings AS book'\
' LEFT JOIN Facilities AS f'\
' ON book.facid = f.facid'\
' WHERE book.memid!=0'\
' GROUP BY f.name'

In [39]:
res = con.execute(query)
res.fetchall()

[('Badminton Court', 344),
 ('Massage Room 1', 421),
 ('Massage Room 2', 27),
 ('Pool Table', 783),
 ('Snooker Table', 421),
 ('Squash Court', 195),
 ('Table Tennis', 385),
 ('Tennis Court 1', 308),
 ('Tennis Court 2', 276)]

Q13: Find the facilities usage by month, but not guests

In [46]:
query = 'SELECT f.name, strftime("%m", book.starttime) AS month, COUNT(*)'\
' FROM Bookings AS book'\
' LEFT JOIN Facilities AS f'\
' ON book.facid = f.facid'\
' WHERE book.memid!=0'\
' GROUP BY f.name, month'

In [47]:
res = con.execute(query)
res.fetchall()

[('Badminton Court', '07', 51),
 ('Badminton Court', '08', 132),
 ('Badminton Court', '09', 161),
 ('Massage Room 1', '07', 77),
 ('Massage Room 1', '08', 153),
 ('Massage Room 1', '09', 191),
 ('Massage Room 2', '07', 4),
 ('Massage Room 2', '08', 9),
 ('Massage Room 2', '09', 14),
 ('Pool Table', '07', 103),
 ('Pool Table', '08', 272),
 ('Pool Table', '09', 408),
 ('Snooker Table', '07', 68),
 ('Snooker Table', '08', 154),
 ('Snooker Table', '09', 199),
 ('Squash Court', '07', 23),
 ('Squash Court', '08', 85),
 ('Squash Court', '09', 87),
 ('Table Tennis', '07', 48),
 ('Table Tennis', '08', 143),
 ('Table Tennis', '09', 194),
 ('Tennis Court 1', '07', 65),
 ('Tennis Court 1', '08', 111),
 ('Tennis Court 1', '09', 132),
 ('Tennis Court 2', '07', 41),
 ('Tennis Court 2', '08', 109),
 ('Tennis Court 2', '09', 126)]